In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from scipy import stats

In [ ]:
def createdata():
  data = {
      'Age': np.random.randint(18, 70, size=20),
      'Salary': np.random.randint(30000, 120000, size=20),
      'Purchased': np.random.choice([0, 1], size=20),
      'Gender': np.random.choice(['Male', 'Female'], size=20),
      'City': np.random.choice(['New York', 'San Francisco', 'Los Angeles'], size=20)
  }

  df = pd.DataFrame(data)
  return df

In [ ]:
df = createdata()
df.head(10)

,Age,Salary,Purchased,Gender,City
0,22,42796,0,Female,San Francisco
1,28,59236,0,Male,San Francisco
2,67,86530,0,Male,New York
3,42,100356,0,Male,New York
4,37,90237,1,Female,San Francisco
5,45,56836,0,Male,Los Angeles
6,37,98245,0,Female,New York
7,49,102731,1,Male,San Francisco
8,45,76583,1,Female,San Francisco
9,43,56878,0,Female,New York


In [ ]:
df.shape

(20, 5)

In [ ]:
# Introduce some missing values for demonstration
df.loc[5, 'Age'] = np.nan
df.loc[10, 'Salary'] = np.nan
df.head(10)

# Basic information about the dataset
print(df.info())

# Summary statistics
print(df.describe())

#Code to Find Missing Values
# Check for missing values in each column
missing_values = df.isnull().sum()

# Display columns with missing values
print(missing_values[missing_values > 0])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Age        19 non-null     float64
 1   Salary     19 non-null     float64
 2   Purchased  20 non-null     int64  
 3   Gender     20 non-null     object 
 4   City       20 non-null     object 
dtypes: float64(2), int64(1), object(2)
memory usage: 932.0+ bytes
None
             Age         Salary  Purchased
count  19.000000      19.000000  20.000000
mean   44.210526   75957.736842   0.450000
std    12.873655   25492.279774   0.510418
min    26.000000   32935.000000   0.000000
25%    36.500000   58082.500000   0.000000
50%    44.000000   78993.000000   0.000000
75%    50.500000   96534.500000   1.000000
max    69.000000  116664.000000   1.000000
Age       1
Salary    1
dtype: int64


In [ ]:
#Set the values to some value (zero, the mean, the median, etc.).
# Step 1: Create an instance of SimpleImputer with the median strategy for Age and mean stratergy for Salary
imputer1 = SimpleImputer(strategy="median")
imputer2 = SimpleImputer(strategy="mean")

df_copy=df

# Step 2: Fit the imputer on the "Age" and "Salary"column
# Note: SimpleImputer expects a 2D array, so we reshape the column
imputer1.fit(df_copy[["Age"]])
imputer2.fit(df_copy[["Salary"]])

# Step 3: Transform (fill) the missing values in the "Age" and "Salary"c column
df_copy["Age"] = imputer1.transform(df[["Age"]])
df_copy["Salary"] = imputer2.transform(df[["Salary"]])

# Verify that there are no missing values left
print(df_copy["Age"].isnull().sum())
print(df_copy["Salary"].isnull().sum())

0
0


In [ ]:
#Handling Categorical Attributes
#Using Ordinal Encoding for gender COlumn and One-Hot Encoding for City Column# Initialize OrdinalEncoder
ordinal_encoder = OrdinalEncoder(categories=[["Male", "Female"]])
# Fit and transform the data
df_copy["Gender_Encoded"] = ordinal_encoder.fit_transform(df_copy[["Gender"]])

# Initialize OneHotEncoder
onehot_encoder = OneHotEncoder()

# Fit and transform the "City" column
encoded_data = onehot_encoder.fit_transform(df[["City"]])

# Convert the sparse matrix to a dense array
encoded_array = encoded_data.toarray()

# Convert to DataFrame for better visualization
encoded_df = pd.DataFrame(encoded_array, columns=onehot_encoder.get_feature_names_out(["City"]))
df_encoded = pd.concat([df_copy, encoded_df], axis=1)

df_encoded.drop("Gender", axis=1, inplace=True)
df_encoded.drop("City", axis=1, inplace=True)

df_encoded. head()



,Age,Salary,Purchased,Gender_Encoded,City_Los Angeles,City_New York,City_San Francisco
0,41.0,102115.0,1,1.0,0.0,0.0,1.0
1,31.0,63941.0,1,1.0,0.0,0.0,1.0
2,53.0,95087.0,0,1.0,0.0,1.0,0.0
3,42.0,33634.0,1,1.0,0.0,1.0,0.0
4,26.0,116664.0,1,0.0,0.0,0.0,1.0


In [ ]:
#Data Transformation
# Min-Max Scaler/Normalization (range 0-1)
#Pros: Keeps all data between 0 and 1; ideal for distance-based models.
#Cons: Can distort data distribution, especially with extreme outliers.
normalizer = MinMaxScaler()
df_encoded[['Salary']] = normalizer.fit_transform(df_encoded[['Salary']])
df_encoded.head()

# Standardization (mean=0, variance=1)
#Pros: Works well for normally distributed data; suitable for many models.
#Cons: Sensitive to outliers.
scaler = StandardScaler()
df_encoded[['Age']] = scaler.fit_transform(df_encoded[['Age']])
df_encoded.head()



,Age,Salary,Purchased,Gender_Encoded,City_Los Angeles,City_New York,City_San Francisco
0,-0.262014,0.826237,1,1.0,0.0,1.0,0.0
1,-1.080806,0.370314,1,1.0,0.0,0.0,1.0
2,0.720537,0.742300,0,1.0,1.0,0.0,0.0
3,-0.180134,0.008348,1,1.0,1.0,0.0,0.0
4,-1.490202,1.000000,1,0.0,0.0,0.0,1.0


In [ ]:
#Removing Outliers
# Outlier Detection and Treatment using IQR.
#Pros: Simple and effective for mild outliers.
#Cons: May overly reduce variation if there are many extreme outliers.
df_encoded_copy1=df_encoded
df_encoded_copy2=df_encoded
df_encoded_copy3=df_encoded

Q1 = df_encoded_copy1['Salary'].quantile(0.25)
Q3 = df_encoded_copy1['Salary'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
df_encoded_copy1['Salary'] = np.where(df_encoded_copy1['Salary'] > upper_bound, upper_bound,
                        np.where(df_encoded_copy1['Salary'] < lower_bound, lower_bound, df_encoded_copy1['Salary']))

df_encoded_copy1.head()

,Age,Salary,Purchased,Gender_Encoded,City_Los Angeles,City_New York,City_San Francisco,Salary_zscore
0,-0.262014,0.826237,1,1.0,0.0,1.0,0.0,1.081589
1,-1.080806,0.370314,1,1.0,0.0,0.0,1.0,-0.496886
2,0.720537,0.742300,0,1.0,1.0,0.0,0.0,0.790985
3,-0.180134,0.008348,1,1.0,1.0,0.0,0.0,-1.750065
4,-1.490202,1.000000,1,0.0,0.0,0.0,1.0,1.683183


In [ ]:
#Removing Outliers
# Z-score method
#Pros: Good for normally distributed data.
#Cons: Not suitable for non-normal data; may miss outliers in skewed distributions.

df_encoded_copy2['Salary_zscore'] = stats.zscore(df_encoded_copy2['Salary'])
df_encoded_copy2['Salary'] = np.where(df_encoded_copy2['Salary_zscore'].abs() > 3, np.nan, df_encoded_copy2['Salary'])  # Replace outliers with NaN
df_encoded_copy2.head()

,Age,Salary,Purchased,Gender_Encoded,City_Los Angeles,City_New York,City_San Francisco,Salary_zscore
0,-0.262014,0.826237,1,1.0,0.0,1.0,0.0,1.081589
1,-1.080806,0.370314,1,1.0,0.0,0.0,1.0,-0.496886
2,0.720537,0.742300,0,1.0,1.0,0.0,0.0,0.790985
3,-0.180134,0.008348,1,1.0,1.0,0.0,0.0,-1.750065
4,-1.490202,1.000000,1,0.0,0.0,0.0,1.0,1.683183


In [ ]:
#Removing Outliers
# Median replacement for outliers
#Pros: Keeps distribution shape intact, useful when capping isn’t feasible.
#Cons: May distort data if outliers represent real phenomena.
df_encoded_copy3['Salary_zscore'] = stats.zscore(df_encoded_copy3['Salary'])
median_salary = df_encoded_copy3['Salary'].median()
df_encoded_copy3['Salary'] = np.where(df_encoded_copy3['Salary_zscore'].abs() > 3, median_salary, df_encoded_copy3['Salary'])
df_encoded_copy3.head()

,Age,Salary,Purchased,Gender_Encoded,City_Los Angeles,City_New York,City_San Francisco,Salary_zscore
0,-0.262014,0.826237,1,1.0,0.0,1.0,0.0,1.081589
1,-1.080806,0.370314,1,1.0,0.0,0.0,1.0,-0.496886
2,0.720537,0.742300,0,1.0,1.0,0.0,0.0,0.790985
3,-0.180134,0.008348,1,1.0,1.0,0.0,0.0,-1.750065
4,-1.490202,1.000000,1,0.0,0.0,0.0,1.0,1.683183


In [ ]:
'''
At the start of the Lab,  in the Observation book, Write python code for the following considering  filename as “housing.csv”
 i. To load .csv file into the data frame
ii. To display information of all columns
iii. To display statistical information of all numerical
iv. To display the count of unique labels for “Ocean Proximity” column
v. To display which attributes (columns) in a dataset have missing values count greater than zero
Step-2: Show the observation book to lab batch faculty incharge.
Step-3: Do the "To Do" tasks given in the PPT
Step-4: At the end of the lab,
i. Write the answers for questions given in the PPT and show it to  lab batch faculty incharge
ii. Should upload the code in your respective GitHub account.
File name format:yourUSN_Lab-1-DataProcessing.ipynb
'''

'\nAt the start of the Lab,  in the Observation book, Write python code for the following considering  filename as “housing.csv”  \n i. To load .csv file into the data frame\nii. To display information of all columns\niii. To display statistical information of all numerical\niv. To display the count of unique labels for “Ocean Proximity” column\nv. To display which attributes (columns) in a dataset have missing values count greater than zero\nStep-2: Show the observation book to lab batch faculty incharge.\nStep-3: Do the "To Do" tasks given in the PPT\nStep-4: At the end of the lab,\ni. Write the answers for questions given in the PPT and show it to  lab batch faculty incharge\nii. Should upload the code in your respective GitHub account.\nFile name format:yourUSN_Lab-1-DataProcessing.ipynb\n\n'

In [ ]:
filename = "/content/housing (1).csv"
df = pd.read_csv(filename)

print("Dataset Information:")
print(df.info())

print("\nStatistical Summary of Numerical Columns:")
print(df.describe())

if "ocean_proximity" in df.columns:
    print("\nUnique Value Counts for 'Ocean Proximity':")
    print(df["ocean_proximity"].value_counts())
else:
    print("\n'Ocean Proximity' column not found in the dataset.")

missing_values = df.isnull().sum()
missing_columns = missing_values[missing_values > 0]

if not missing_columns.empty:
    print("\nColumns with Missing Values:")
    print(missing_columns)
else:
    print("\nNo missing values found in the dataset.")


Dataset Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB
None

Statistical Summary of Numerical Columns:
          longitude      latitude  housing_median_age   total_rooms  \
count  20640.000000  20640.000000        20640.000000  20640.000000   
mean    -119.569704     35.631861       

In [ ]:
data2 = pd.read_csv("/content/Dataset_with_Nulls.csv")
data2.head()

,ID,No_Pation,Gender,AGE,Urea,Cr,HbA1c,Chol,TG,HDL,LDL,VLDL,BMI,CLASS
0,NaN,NaN,F,50.0,4.7,46.0,4.9,4.2,0.9,2.4,1.4,0.5,24.0,N
1,735.0,34221.0,M,26.0,4.5,62.0,4.9,3.7,1.4,1.1,2.1,0.6,NaN,N
2,420.0,47975.0,F,50.0,4.7,46.0,4.9,4.2,0.9,2.4,1.4,0.5,24.0,N
3,680.0,87656.0,F,50.0,4.7,46.0,4.9,4.2,NaN,2.4,1.4,0.5,24.0,N
4,504.0,NaN,M,33.0,7.1,46.0,4.9,4.9,1.0,NaN,2.0,0.4,21.0,N


In [ ]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   ID         900 non-null    float64
 1   No_Pation  902 non-null    float64
 2   Gender     914 non-null    object 
 3   AGE        889 non-null    float64
 4   Urea       902 non-null    float64
 5   Cr         906 non-null    float64
 6   HbA1c      905 non-null    float64
 7   Chol       908 non-null    float64
 8   TG         895 non-null    float64
 9   HDL        909 non-null    float64
 10  LDL        909 non-null    float64
 11  VLDL       908 non-null    float64
 12  BMI        892 non-null    float64
 13  CLASS      917 non-null    object 
dtypes: float64(12), object(2)
memory usage: 109.5+ KB


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler, MinMaxScaler
from scipy import stats

# Load dataset
file_path = "/content/Dataset_with_Nulls.csv"
df = pd.read_csv(file_path)

### Step 1: Handling Missing Values ###
# Identify missing values
print("Missing values before handling:\n", df.isnull().sum())

# Handling missing numerical columns - Median for 'Age', Mean for other numerical values
num_imputer_median = SimpleImputer(strategy="median")
num_imputer_mean = SimpleImputer(strategy="mean")

df["AGE"] = num_imputer_median.fit_transform(df[["AGE"]])
for col in ["Urea", "Cr", "HbA1c", "Chol", "TG", "HDL", "LDL", "VLDL", "BMI"]:
    df[col] = num_imputer_mean.fit_transform(df[[col]])

# Convert categorical columns to string type
df["Gender"] = df["Gender"].astype(str)
df["CLASS"] = df["CLASS"].astype(str)

# Handling missing categorical columns - Fill with Mode
cat_imputer = SimpleImputer(strategy="most_frequent")

df["Gender"] = cat_imputer.fit_transform(df[["Gender"]]).ravel()
df["CLASS"] = cat_imputer.fit_transform(df[["CLASS"]]).ravel()

print("Missing values after handling:\n", df.isnull().sum())

### Step 2: Handling Categorical Attributes ###
# Encode 'Gender' using Ordinal Encoding (Male = 0, Female = 1)
ordinal_encoder = OrdinalEncoder(categories=[["Male", "Female"]])
df['Gender'] = df['Gender'].replace({'F': 'Female', 'f': 'Female', 'M': 'Male'})  # Handle variations
# Replace NaN with Mode (most frequent) for Gender
df['Gender'] = df['Gender'].fillna(df['Gender'].mode()[0]) # Fill na with mode if any

# Now apply ordinal encoding
ordinal_encoder = OrdinalEncoder(categories=[["Male", "Female"]])
# One-Hot Encoding for 'CLASS'
onehot_encoder = OneHotEncoder(sparse_output=False)
class_encoded = onehot_encoder.fit_transform(df[["CLASS"]])

# Convert to DataFrame
class_encoded_df = pd.DataFrame(class_encoded, columns=onehot_encoder.get_feature_names_out(["CLASS"]))

# Merge One-Hot Encoded Data and drop original categorical columns
df = pd.concat([df, class_encoded_df], axis=1)
df.drop(["Gender", "CLASS"], axis=1, inplace=True)

### Step 3: Data Transformation ###
# Min-Max Scaling for Salary
minmax_scaler = MinMaxScaler()
df[["Urea", "Cr", "HbA1c", "Chol", "TG", "HDL", "LDL", "VLDL", "BMI"]] = minmax_scaler.fit_transform(
    df[["Urea", "Cr", "HbA1c", "Chol", "TG", "HDL", "LDL", "VLDL", "BMI"]]
)

# Standardization for Age
standard_scaler = StandardScaler()
df[["AGE"]] = standard_scaler.fit_transform(df[["AGE"]])

### Step 4: Removing Outliers ###
# IQR Method for 'Salary' (Replacing Outliers with Boundaries)
for col in ["Urea", "Cr", "HbA1c", "Chol", "TG", "HDL", "LDL", "VLDL", "BMI"]:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df[col] = np.where(df[col] > upper_bound, upper_bound, np.where(df[col] < lower_bound, lower_bound, df[col]))

# Z-score method for 'AGE' (Replacing Outliers with NaN)
df["AGE_zscore"] = stats.zscore(df["AGE"])
df["AGE"] = np.where(df["AGE_zscore"].abs() > 3, np.nan, df["AGE"])

# Median Replacement for Outliers in 'AGE'
median_age = df["AGE"].median()
df["AGE"] = np.where(df["AGE"].isnull(), median_age, df["AGE"])

# Drop auxiliary columns
df.drop(columns=["AGE_zscore"], inplace=True)

df

print("Preprocessing Complete. Cleaned dataset saved!")


Missing values before handling:
 ID           100
No_Pation     98
Gender        86
AGE          111
Urea          98
Cr            94
HbA1c         95
Chol          92
TG           105
HDL           91
LDL           91
VLDL          92
BMI          108
CLASS         83
dtype: int64
Missing values after handling:
 ID           100
No_Pation     98
Gender         0
AGE            0
Urea           0
Cr             0
HbA1c          0
Chol           0
TG             0
HDL            0
LDL            0
VLDL           0
BMI            0
CLASS          0
dtype: int64
Preprocessing Complete. Cleaned dataset saved!
